# Matrix Exponential Attempt

Trying to calculate the matrix exponent

In [8]:
import numpy as np
import scipy
from itertools import product

In [ ]:
def pick_pade_structure(Am):
	Py_ssize_t n = Am.shape[1]
	lm = 0
	s = 0
	intn = n
	n2 = intn * intn
	int_one = 1
	Amv = np.asarray(Am)
	one = 1.0
	zero = 0.0
	u = (2.0)**-53
	absA = np.zeros(n)
	work = np.zeros(n)
	work2 = np.zeros(n)
	
	theta = np.array([1.495585217958292e-002,
					  2.539398330063230e-001,
					  9.504178996162932e-001,
					  2.097847961257068e+000,
					  4.250000000000000e+000])
	coeff = np.array([u*100800.0,
					  u*10059033600.0,
					  u*4487938430976000.0,
					  u*5914384781877411840000.0,
					  u*113250775606021113483283660800000000.0])
	try:
		for j in range(n):
			work[j] = 1.
		for i in range(n):
			for j in range(n):
				absA[i*n + j] = abs(Am[0, i, j])

		# First spin = normest(|A|, 1), increase m when spun more
		normA = kth_power_norm_d(absA, work, work2, n, 1)
		{{ pfx }}gemm(cN, cN, &intn, &intn, &intn, &one, &Amv[0, 0, 0], &intn, &Amv[0, 0, 0], &intn, &zero, &Amv[1, 0, 0], &intn)
		{{ pfx }}gemm(cN, cN, &intn, &intn, &intn, &one, &Amv[1, 0, 0], &intn, &Amv[1, 0, 0], &intn, &zero, &Amv[2, 0, 0], &intn)
		{{ pfx }}gemm(cN, cN, &intn, &intn, &intn, &one, &Amv[2, 0, 0], &intn, &Amv[1, 0, 0], &intn, &zero, &Amv[3, 0, 0], &intn)
		d4 = norm1(Amv[2]) ** (1./4.)
		d6 = norm1(Amv[3]) ** (1./6.)
		eta0 = max(d4, d6)
		eta1 = eta0

		# m = 3
		temp = kth_power_norm_d(absA, work, work2, n, 6)
		lm = max(<int>ceil(log2(temp/normA/coeff[0])/6), 0)
		if eta0 < theta[0] and lm == 0:
			return 3, s

		# m = 5
		temp = kth_power_norm_d(absA, work, work2, n, 4)
		lm = max(<int>ceil(log2(temp/normA/coeff[1])/10), 0)
		if eta1 < theta[1] and lm == 0:
			return 5, s

		# m = 7
		if n < 400:
			{{ pfx }}gemm(cN, cN, &intn, &intn, &intn, &one, &Amv[2, 0, 0], &intn, &Amv[2, 0, 0], &intn, &zero, &Amv[4, 0, 0], &intn)
			d8 = norm1(Amv[4, :, :]) ** (1./8.)
		else:
			d8  = _norm1est(np.asarray(Am[0]), m=8) ** (1./8.)

		eta2 = max(d6, d8)
		temp = kth_power_norm_d(absA, work, work2, n, 4)
		lm = max(<int>ceil(log2(temp/normA/coeff[2])/14), 0)
		if eta2 < theta[2] and lm == 0:
			return 7, s

		# m = 9
		temp = kth_power_norm_d(absA, work, work2, n, 4)
		lm = max(<int>ceil(log2(temp/normA/coeff[3])/18), 0)
		if eta2 < theta[3] and lm == 0:
			return 9, s

		# m = 13
		# Scale-square
		if n < 400:
			{{ pfx }}gemm(cN, cN, &intn, &intn, &intn, &one, &Amv[3, 0, 0], &intn, &Amv[2, 0, 0], &intn, &zero, &Amv[4, 0, 0], &intn)
			d10 = norm1(Amv[4, :, :]) ** (1./10.)
		else:
			d10  = _norm1est(np.asarray(Am[0]), m=10) ** (1./10.)

		eta3 = max(d8, d10)
		eta4 = min(eta2, eta3)
		s = max(<int>ceil(log2(eta4/theta[4])), 0)
		if s != 0:
			two_pow_s = 2.** (-s)
			dscal(&n2, &two_pow_s, absA, &int_one)
			# kth_power_norm has spun 19 times already
			two_pow_s = 2.** ((-s)*19.)
			for i in range(n):
				work[i] *= two_pow_s
			normA *= 2.**(-s)
		temp = kth_power_norm_d(absA, work, work2, n, 8)
		s += max(<int>ceil(log2(temp/normA/coeff[4])/26), 0)
		return 13, s
	finally:
		free(work)
		free(work2)
		free(absA)


In [20]:
def ScipyVersion(A):
	a = np.asarray(A)

	# larger problem with unspecified stacked dimensions.
	n = a.shape[-1]
	eA = np.empty(a.shape, dtype=a.dtype)
	# working memory to hold intermediate arrays
	Am = np.empty((5, n, n), dtype=a.dtype)

	# Main loop to go through the slices of an ndarray and passing to expm
	for ind in product(*[range(x) for x in a.shape[:-2]]):
		aw = a[ind]
		# Generic/triangular case; copy the slice into scratch and send.
		# Am will be mutated by pick_pade_structure
		Am[0, :, :] = aw
		m, s = pick_pade_structure(Am)

		if s != 0:  # scaling needed
			Am[:4] *= [[[2**(-s)]], [[4**(-s)]], [[16**(-s)]], [[64**(-s)]]]

		pade_UV_calc(Am, n, m)
		eAw = Am[0]
		for _ in range(s):
			eAw = eAw @ eAw

		# Zero out the entries from np.empty in case of triangular input
		eA[ind] = eAw

	return eA

In [21]:
matrix = np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 0]])

In [22]:
ScipyVersion(matrix)

NameError: name 'pick_pade_structure' is not defined

In [12]:
scipy.linalg.bandwidth(matrix)

(1, 1)